In [5]:
!wget https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv

--2022-09-19 18:35:11--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/housing.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1423529 (1.4M) [text/plain]
Saving to: ‘housing.csv’

housing.csv         100%[===================>]   1.36M  --.-KB/s    in 0.05s   

2022-09-19 18:35:11 (24.9 MB/s) - ‘housing.csv’ saved [1423529/1423529]



In [6]:
import pandas as pd 
housing = pd.read_csv("/content/housing.csv")

Select only relevant columns 

In [7]:
hdf=housing[['latitude',
'longitude',
'housing_median_age',
'total_rooms',
'total_bedrooms',
'population',
'households',
'median_income',
'median_house_value']]

1. How many nulls in data set 
207

In [8]:
hdf.isnull().sum() #207 

latitude                0
longitude               0
housing_median_age      0
total_rooms             0
total_bedrooms        207
population              0
households              0
median_income           0
median_house_value      0
dtype: int64

2.median for variable population 
1166

In [9]:
hdf.population.median()

1166.0

Shuffle the initail datset using 42 seed 

In [ ]:

# hdf= hdf.sample(frac=1, random_state=42)

In [10]:
import numpy as np 

In [11]:
n=len(hdf)


In [ ]:
n-(2*4128)

In [12]:
nval=int(0.2*n)
ntest=int(0.2*n)
ntrain=n-nval-ntest

In [13]:
l=list(range(n))
np.random.seed(42)
np.random.shuffle(l)
hdf=hdf.iloc[l]

In [14]:
hdf_train=hdf.iloc[:ntrain]
hdf_val=hdf.iloc[ntrain:ntrain+nval]
hdf_test=hdf.iloc[ntrain+nval:]


In [15]:
hdfval= hdf.iloc[ntrain:ntrain+nval]
hdftest= hdf.iloc[nval:nval+ntest]

In [16]:
hdf_ytr=hdf_train["median_house_value"]

hdf_yval=hdf_val["median_house_value"]

hdf_ytest=hdf_test["median_house_value"]




In [17]:
hdf_train.drop(columns="median_house_value",inplace=True)
hdf_val.drop(columns="median_house_value",inplace=True)
hdf_test.drop(columns="median_house_value",inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4913: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [18]:
hdf_ytrlg=np.log1p(hdf_ytr)
hdf_yvallg=np.log1p(hdf_yval)
hdf_ytestlg=np.log1p(hdf_ytest)

Q3  fill the missing with 0 or mean 
for  null  coloumn

In [19]:
hdf_train=hdf_train.reset_index(drop=True)
hdf_val=hdf_val.reset_index(drop=True)
hdf_test=hdf_test.reset_index(drop=True)

In [20]:
hdf_train_filledmean=hdf_train.fillna(hdf_train.total_bedrooms.mean())
hdf_val_filledmean=hdf_val.fillna(hdf_train.total_bedrooms.mean())
hdf_test_filledmean=hdf_test.fillna(hdf_train.total_bedrooms.mean())


In [21]:
hdf_train_filledz=hdf_train.fillna(0)
hdf_val_filledz=hdf_val.fillna(0)
hdf_test_filledz=hdf_test.fillna(0)


In [30]:
check=list(hdf_train.columns)
check

['latitude',
 'longitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income']

In [31]:
def prepare_X(df, fillna_value):
    df_num = df[check]
    df_num = df_num.fillna(fillna_value)
    X = df_num.values
    return X

In [32]:
def linearreg(X,y):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])

    XTX = X.T.dot(X)
    # reg = r * np.eye(XTX.shape[0])
    # XTX = XTX + reg

    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    
    return w[0], w[1:]
def rmse(y, y_pred):
    error = y_pred - y
    mse = (error ** 2).mean()
    return np.sqrt(mse)

In [26]:
mean=hdf_train.total_bedrooms.mean()

In [42]:
hdf_train_values=prepare_X(hdf_train,fillna_value=mean)
wmean0,wmean=linearreg(hdf_train_values,hdf_ytrlg)


In [48]:
hdf_val_values_mean=prepare_X(hdf_val,fillna_value=mean)
y_val_meanpred=wmean0+hdf_val_values_mean.dot(wmean)


In [49]:
np.round(rmse(hdf_yvallg,y_val_meanpred),2)

0.33

In [50]:
hdf_train_values=prepare_X(hdf_train,fillna_value=0)
wnull0,wnull=linearreg(hdf_train_values,hdf_ytrlg)
hdf_val_values_null=prepare_X(hdf_val,fillna_value=0)
y_val_nullpred=wnull0+hdf_val_values_null.dot(wnull)


In [51]:
np.round(rmse(hdf_yvallg,y_val_nullpred),2)

0.33

Same values observed both equally good 

In [41]:
#4th question

In [52]:
# Regularization
def linearreg(X,y,r=0.01):
    ones = np.ones(X.shape[0])
    X = np.column_stack([ones, X])
    XTX = X.T.dot(X)
    reg = r * np.eye(XTX.shape[0])
    XTX = XTX + reg
    XTX_inv = np.linalg.inv(XTX)
    w = XTX_inv.dot(X.T).dot(y)
    return w[0], w[1:]

for r in [0, 0.000001, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10]:
    w_0, w = linearreg(hdf_train_values, hdf_ytrlg, r=r)
    y_null_reg_val = w_0 + hdf_val_values_null.dot(w)
    rmse_val = np.round(rmse(hdf_yvallg,y_null_reg_val ),2)
    print(r, w_0, rmse_val)


0 -11.686975241808469 0.33
1e-06 -11.686959175857494 0.33
0.0001 -11.685368865259061 0.33
0.001 -11.670931318158033 0.33
0.01 -11.528493585707224 0.33
0.1 -10.274500281964695 0.33
1 -4.920480897816711 0.33
5 -1.4820957455971928 0.34
10 -0.7899311832023754 0.34


1st is fine r=0